## Yelp Data Analysis - Part 2 - Data Cleansing & Wrangling
Cleaning list:<br>
* Split ranking and name
* Make ranking the index
* Change rating to only integers and cast data type to integer
* Add another column to translate dollar signs to actual value. Figure out what to do with "N/A" values.
* Clean categories column and find a way to separate categories<br>

### Import libraries and CSV file:

In [1]:
import numpy as np
import pandas as pd

df = pd.read_csv('rva_restaurants_scraped.csv')
df.head(5)

,name,rating,price,review_count,categories
0,1. Toast,4 star rating,$$,619,"['Southern, ', 'Gastropubs']"
1,2. Lucky AF,5 star rating,NaN,9,"['Asian Fusion, ', 'Sushi Bars, ', 'Pan Asian']"
2,3. Stella’s,4.5 star rating,$$,715,"['Mediterranean, ', 'Greek, ', 'Cocktail Bars']"
3,4. Secret Sandwich Society,4.5 star rating,$$,936,"['Burgers, ', 'Sandwiches, ', 'Cocktail Bars']"
4,5. Laura Lee’s,4.5 star rating,$$,260,['American (New)']


### Clean 'name' column:

Split ranking and name into separate columns:<br>
https://www.geeksforgeeks.org/split-a-text-column-into-two-columns-in-pandas-dataframe/

In [2]:
# Use lambda expression/.apply() to split string
df[['rank', 'name']] = df['name'].apply(lambda x: pd.Series(str(x).split('.')[0:2]))
df.head()

,name,rating,price,review_count,categories,rank
0,Toast,4 star rating,$$,619,"['Southern, ', 'Gastropubs']",1
1,Lucky AF,5 star rating,NaN,9,"['Asian Fusion, ', 'Sushi Bars, ', 'Pan Asian']",2
2,Stella’s,4.5 star rating,$$,715,"['Mediterranean, ', 'Greek, ', 'Cocktail Bars']",3
3,Secret Sandwich Society,4.5 star rating,$$,936,"['Burgers, ', 'Sandwiches, ', 'Cocktail Bars']",4
4,Laura Lee’s,4.5 star rating,$$,260,['American (New)'],5


### Reordering columns; changing 'rank' type to int; setting 'rank' as integer index label:

In [3]:
# Reorder by simply calling dataframe with difference column orders
df = df[['rank','name','rating','price','review_count','categories']]
df.head()

,rank,name,rating,price,review_count,categories
0,1,Toast,4 star rating,$$,619,"['Southern, ', 'Gastropubs']"
1,2,Lucky AF,5 star rating,NaN,9,"['Asian Fusion, ', 'Sushi Bars, ', 'Pan Asian']"
2,3,Stella’s,4.5 star rating,$$,715,"['Mediterranean, ', 'Greek, ', 'Cocktail Bars']"
3,4,Secret Sandwich Society,4.5 star rating,$$,936,"['Burgers, ', 'Sandwiches, ', 'Cocktail Bars']"
4,5,Laura Lee’s,4.5 star rating,$$,260,['American (New)']


In [4]:
# Change 'rank' data type from string to int
df['rank'] = df['rank'].astype(int)

In [5]:
# Set 'rank' column as the dataframe index
df.set_index('rank', inplace=True)
df.head()

,name,rating,price,review_count,categories
rank,,,,,
1,Toast,4 star rating,$$,619,"['Southern, ', 'Gastropubs']"
2,Lucky AF,5 star rating,NaN,9,"['Asian Fusion, ', 'Sushi Bars, ', 'Pan Asian']"
3,Stella’s,4.5 star rating,$$,715,"['Mediterranean, ', 'Greek, ', 'Cocktail Bars']"
4,Secret Sandwich Society,4.5 star rating,$$,936,"['Burgers, ', 'Sandwiches, ', 'Cocktail Bars']"
5,Laura Lee’s,4.5 star rating,$$,260,['American (New)']


### The 'name' column still keeps that weird substring "\xa0" before every restaurant name. Let's get rid of that with a function:
Note: This weird substring is not visible when you "print" the column, row call. However this affects if you want to use booleans to find a match.

In [6]:
df['name'].loc[1]

'\xa0Toast'

In [7]:
# Nothing will come up
df[df['name']=='Toast']

,name,rating,price,review_count,categories
rank,,,,,


Write function cleanName() to edit for patterns in name string. Use .apply() and lambda functions:

In [8]:
def cleanName(x):
    if '\xa0' in x:
        x = x.replace('\xa0','')
        return x

df['name'] = df['name'].apply(lambda x: cleanName(x))

### Previous use of pure Python for loop

# for i in range(1, len(df['name'])+1):    
#     if '\xa0' in df['name'].loc[i]:
#         df['name'].loc[i] = df['name'].loc[i].replace('\xa0','')

# df['name'].loc[1]

Run some test scripts:

In [9]:
df['name'].loc[50]

'Tazza Kitchen Scott’s Addition'

In [10]:
df[df['name'] =='Toast']

,name,rating,price,review_count,categories
rank,,,,,
1,Toast,4 star rating,$$,619,"['Southern, ', 'Gastropubs']"


In [11]:
df['name'].loc[45]

'Tres Machos'

In [12]:
df[df['name']=='Tres Machos']

,name,rating,price,review_count,categories
rank,,,,,
45,Tres Machos,4 star rating,$$,89,['Mexican']


### Convert the star rating column to *just* the integer so we can quantify later:

In [13]:
def cleanRatings(rating):
    if ' star rating' in rating:
        rating = rating.replace(' star rating' , '')
        return rating
    
df['rating'] = df['rating'].apply(lambda rating: cleanRatings(rating))

# Change data type to integer
df['rating'] = df['rating'].astype(float)
df.head()

# Previous pure Python for loop
# for i in range(1, len(df['rating'])+1):    
#     if ' star rating' in df['rating'].loc[i]:
#         df['rating'].loc[i] = df['rating'].loc[i].replace(' star rating','')

,name,rating,price,review_count,categories
rank,,,,,
1,Toast,4.0,$$,619,"['Southern, ', 'Gastropubs']"
2,Lucky AF,5.0,NaN,9,"['Asian Fusion, ', 'Sushi Bars, ', 'Pan Asian']"
3,Stella’s,4.5,$$,715,"['Mediterranean, ', 'Greek, ', 'Cocktail Bars']"
4,Secret Sandwich Society,4.5,$$,936,"['Burgers, ', 'Sandwiches, ', 'Cocktail Bars']"
5,Laura Lee’s,4.5,$$,260,['American (New)']


### Can't quantify and operate on dollar signs. Convert them into a normalized numerical scale (1-4):
I'll create a function and map/apply it to the df['price'] column.
<br><br>
Note: we cannot translate dollar signs to exact prices as each number of signs translates to *range* of meal prices per person (incl. tax, tips, etc).
* 1 = under 10 dollars
* 2 = 11-30 dollars
* 3 = 31-60 dollars
* 4 = over 61 dollars

In [14]:
# Create function to translate dollar signs into normalized price ranges 1-4.
def priceTranslator(price):
    '''
    Input df['price'] via lambda and this will convert dollar signs into normalized integers that we can operate on.\
    We can't put exact prices because each dollar sign signifies a range of prices. We will scale 1 to 4.
    '''
    try:
        if price == '$':
            return 1
        elif price == '$$':
            return 2
        if price == '$$$':
            return 3
        if price == '$$$$':
            return 4
    except:
        pass

In [15]:
# Applied lambda function
df['priceIndex'] = df['price'].apply(lambda x: priceTranslator(x))
# Reorder columns
df = df[['name','rating','price','priceIndex','review_count','categories']]
df.head()

,name,rating,price,priceIndex,review_count,categories
rank,,,,,,
1,Toast,4.0,$$,2.0,619,"['Southern, ', 'Gastropubs']"
2,Lucky AF,5.0,NaN,NaN,9,"['Asian Fusion, ', 'Sushi Bars, ', 'Pan Asian']"
3,Stella’s,4.5,$$,2.0,715,"['Mediterranean, ', 'Greek, ', 'Cocktail Bars']"
4,Secret Sandwich Society,4.5,$$,2.0,936,"['Burgers, ', 'Sandwiches, ', 'Cocktail Bars']"
5,Laura Lee’s,4.5,$$,2.0,260,['American (New)']


In [16]:
df.rename(columns = {'review_count':'reviewCount'}, inplace=True)
df.head()

,name,rating,price,priceIndex,reviewCount,categories
rank,,,,,,
1,Toast,4.0,$$,2.0,619,"['Southern, ', 'Gastropubs']"
2,Lucky AF,5.0,NaN,NaN,9,"['Asian Fusion, ', 'Sushi Bars, ', 'Pan Asian']"
3,Stella’s,4.5,$$,2.0,715,"['Mediterranean, ', 'Greek, ', 'Cocktail Bars']"
4,Secret Sandwich Society,4.5,$$,2.0,936,"['Burgers, ', 'Sandwiches, ', 'Cocktail Bars']"
5,Laura Lee’s,4.5,$$,2.0,260,['American (New)']


### Edit the 'categories' column to look cleaner:
* Remove excess commas and string brackets.
* Then convert it to an actual list. (Note: the final result will "look" the same but we will be able iterate through it)
* Use `.explode('categories')` to unnest 'categories' column which is a column of lists
* Note: We will need to clean it when we reupload it to another notebook.

We need to clean the strings in this column first:<br>
* Let's remove the excess commas and quotation marks
* CLEAN UP THIS METHOD!

In [17]:
df.head()

,name,rating,price,priceIndex,reviewCount,categories
rank,,,,,,
1,Toast,4.0,$$,2.0,619,"['Southern, ', 'Gastropubs']"
2,Lucky AF,5.0,NaN,NaN,9,"['Asian Fusion, ', 'Sushi Bars, ', 'Pan Asian']"
3,Stella’s,4.5,$$,2.0,715,"['Mediterranean, ', 'Greek, ', 'Cocktail Bars']"
4,Secret Sandwich Society,4.5,$$,2.0,936,"['Burgers, ', 'Sandwiches, ', 'Cocktail Bars']"
5,Laura Lee’s,4.5,$$,2.0,260,['American (New)']


In [24]:
def cleanCategories(list, dict):
    '''
    Clean categories of excess elements, then will convert to list off of commas
    list = element; dict = dictionary of {old, new}
    '''
    for key in dict.keys():
        list = list.replace(key, dict[key])
    list = list.split(", ")
    return list

replace_values = {"['" : "", "']" : "", ", ', '": ", "}

df['categories'].apply(lambda list: cleanCategories(list, replace_values))

AttributeError: 'list' object has no attribute 'replace'

In [21]:
df['categories'] = df['categories'].apply(lambda list: cleanCategories(list, replace_values))

In [22]:
df.head()

,name,rating,price,priceIndex,reviewCount,categories
rank,,,,,,
1,Toast,4.0,$$,2.0,619,"[Southern, Gastropubs]"
2,Lucky AF,5.0,NaN,NaN,9,"[Asian Fusion, Sushi Bars, Pan Asian]"
3,Stella’s,4.5,$$,2.0,715,"[Mediterranean, Greek, Cocktail Bars]"
4,Secret Sandwich Society,4.5,$$,2.0,936,"[Burgers, Sandwiches, Cocktail Bars]"
5,Laura Lee’s,4.5,$$,2.0,260,[American (New)]


In [25]:
df['categories'].loc[2][1]

'Sushi Bars'

In [77]:
# df['categories'] = df['categories'].apply(lambda x: x.replace(", '," , ""))
# df['categories'] = df['categories'].apply(lambda x: x.replace(" '", ", "))
# df['categories'] = df['categories'].apply(lambda x: x.replace("'", ""))

# def stringCleaner(x):
#     if "[" or "]" in x:
#         x = x.replace("[","")
#         x = x.replace("]","")
#     return x

# df['categories'] = df['categories'].apply(lambda x: stringCleaner(x))

# def categoriesToList(x):
#     x = x.split(", ")
#     return x

# df['categories'] = df['categories'].apply(lambda x: categoriesToList(x))

### Looks like our scraper picked up duplicate restaurants. Let's clean that up.
It could be because Yelp is constantly changing the rankings of restaurants in real time. 
We'll use `.drop_duplicate()`

In [26]:
df.head()

,name,rating,price,priceIndex,reviewCount,categories
rank,,,,,,
1,Toast,4.0,$$,2.0,619,"[Southern, Gastropubs]"
2,Lucky AF,5.0,NaN,NaN,9,"[Asian Fusion, Sushi Bars, Pan Asian]"
3,Stella’s,4.5,$$,2.0,715,"[Mediterranean, Greek, Cocktail Bars]"
4,Secret Sandwich Society,4.5,$$,2.0,936,"[Burgers, Sandwiches, Cocktail Bars]"
5,Laura Lee’s,4.5,$$,2.0,260,[American (New)]


In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 50 entries, 1 to 50
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   name         50 non-null     object 
 1   rating       50 non-null     float64
 2   price        45 non-null     object 
 3   priceIndex   45 non-null     float64
 4   reviewCount  50 non-null     int64  
 5   categories   50 non-null     object 
dtypes: float64(2), int64(1), object(3)
memory usage: 5.2+ KB


In [28]:
df.drop_duplicates(subset=['name'], inplace=True)

In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 40 entries, 1 to 50
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   name         40 non-null     object 
 1   rating       40 non-null     float64
 2   price        36 non-null     object 
 3   priceIndex   36 non-null     float64
 4   reviewCount  40 non-null     int64  
 5   categories   40 non-null     object 
dtypes: float64(2), int64(1), object(3)
memory usage: 2.2+ KB


### Fill in missing prices:

In [30]:
#Manual fill of restaurants missing prices

df.loc[22, 'price'] = '$$$' # The Broken Tulip
df.loc[22, 'priceIndex'] = 3.0

df.loc[15, 'price'] = '$$' # &pizza
df.loc[15, 'priceIndex'] = 2.0

df.loc[2, 'price'] = '$$' # Lucky AF
df.loc[2, 'priceIndex'] = 2.0

df.loc[16, 'price'] = '$$$' # Perch
df.loc[16, 'priceIndex'] = 3.0

In [31]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 40 entries, 1 to 50
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   name         40 non-null     object 
 1   rating       40 non-null     float64
 2   price        40 non-null     object 
 3   priceIndex   40 non-null     float64
 4   reviewCount  40 non-null     int64  
 5   categories   40 non-null     object 
dtypes: float64(2), int64(1), object(3)
memory usage: 3.4+ KB


We now only have 40 rows after cleaning up for duplicates, but we can work with this.

### We will now create a seperate dataframe with the nest list of categories expand for extended analyses
* We will use the .explode() method and reset the index
* Note: We will not save this into csv. We will just re-explode during analysis to save file space.

Copy dataframe to df2 for testing:

In [32]:
df2 = df.copy(deep=True)

https://stackoverflow.com/questions/53218931/how-to-unnest-explode-a-column-in-a-pandas-dataframe

In [34]:
df2 = df2.explode('categories')
df2.reset_index(inplace = True)
df2

,rank,name,rating,price,priceIndex,reviewCount,categories
0,1,Toast,4.0,$$,2.0,619,Southern
1,1,Toast,4.0,$$,2.0,619,Gastropubs
2,2,Lucky AF,5.0,$$,2.0,9,Asian Fusion
3,2,Lucky AF,5.0,$$,2.0,9,Sushi Bars
4,2,Lucky AF,5.0,$$,2.0,9,Pan Asian
...,...,...,...,...,...,...,...
74,48,Beijing On Grove,4.0,$$,2.0,125,Bars
75,48,Beijing On Grove,4.0,$$,2.0,125,Cantonese
76,50,Tazza Kitchen Scott’s Addition,4.0,$$,2.0,193,American (New)
77,50,Tazza Kitchen Scott’s Addition,4.0,$$,2.0,193,Pizza


In [35]:
df2['categories'].value_counts().head(10)

American (New)            9
Southern                  8
Seafood                   5
Bars                      5
Cocktail Bars             5
Burgers                   4
Mediterranean             4
Pizza                     3
American (Traditional)    2
Brewpubs                  2
Name: categories, dtype: int64

In [36]:
df2[df2['categories'] == 'Southern'][['rank','name']]

,rank,name
0,1,Toast
14,6,Lunch Or Supper
15,7,Southbound
23,13,The Roosevelt
40,23,Croaker’s Spot Restaurant
63,43,Sugar’s Crab Shack
69,46,Spoonbread Bistro
72,47,Soul Taco


In [46]:
df.head()

,name,rating,price,priceIndex,reviewCount,categories
rank,,,,,,
1,Toast,4.0,$$,2.0,619,"[Southern, Gastropubs]"
2,Lucky AF,5.0,NaN,NaN,9,"[Asian Fusion, Sushi Bars, Pan Asian]"
3,Stella’s,4.5,$$,2.0,715,"[Mediterranean, Greek, Cocktail Bars]"
4,Secret Sandwich Society,4.5,$$,2.0,936,"[Burgers, Sandwiches, Cocktail Bars]"
5,Laura Lee’s,4.5,$$,2.0,260,[American (New)]


In [47]:
df2.head()

,rank,name,rating,price,priceIndex,reviewCount,categories
0,1,Toast,4.0,$$,2.0,619,Southern
1,1,Toast,4.0,$$,2.0,619,Gastropubs
2,2,Lucky AF,5.0,NaN,NaN,9,Asian Fusion
3,2,Lucky AF,5.0,NaN,NaN,9,Sushi Bars
4,2,Lucky AF,5.0,NaN,NaN,9,Pan Asian


### Save on to CSV file:

In [37]:
df.to_csv('/Users/terencerustia/Documents/Data Science Projects/Yelp Data Analysis/rva_restaurants_cleaned_df.csv',)

Upon upload to next notebook, we'll need to re-wrangle the data and set up df2 to un-nest the categories column again.